In [ ]:
!pip install transformers
!pip install accelerate
!pip install OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import pickle, sys, copy, re, math
import pandas as pd
import concurrent.futures
from collections import Counter
import numpy as np
import time


In [ ]:
#mounting google drive to avoid uploading files each runtime
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
#Read the csv dataset as a pandas dataframe
path='/content/drive/MyDrive/Data/claudette_train_multi_label.csv'
df_train = pd.read_csv(path)
#Print to check if this is the correct dataset
df_train.head()

Mounted at /content/drive


,Index,Document,Labels,Sentence
0,0,Mozilla,[],websites & communications terms of use
1,1,Mozilla,[],please read the terms of this entire document ...
2,2,Mozilla,['USE'],by accessing or signing up to receive communic...
3,3,Mozilla,[],our websites include multiple domains such as ...
4,4,Mozilla,[],you may also recognize our websites by nicknam...


In [ ]:
#Read the csv dataset as a pandas dataframe
path='/content/drive/MyDrive/Data/claudette_val_multi_label.csv'
df_val = pd.read_csv(path)
#Print to check if this is the correct dataset
df_val.head()


,Index,Document,Labels,Sentence
0,72,MyHeritage,[],myheritage - terms and conditionswelcome to my...
1,73,MyHeritage,[],myheritage is an online service that allows me...
2,74,MyHeritage,[],the service is owned and operated by myheritag...
3,75,MyHeritage,[],"` you ' or ` your ' means an adult user , over..."
4,76,MyHeritage,[],any use of the service and/or the myheritage w...


In [ ]:
#Read the csv dataset as a pandas dataframe
path='/content/drive/MyDrive/Data/claudette_test_multi_label.csv'
df_test = pd.read_csv(path)
#Print to check if this is the correct dataset
df_test.head()

,Index,Document,Labels,Sentence
0,992,Pinterest,[],thank you for using pinterest !
1,993,Pinterest,[],these terms of service -lrb- `` terms '' -rrb-...
2,994,Pinterest,[],"please read these terms carefully , and contac..."
3,995,Pinterest,"['PINC', 'USE']","by accessing or using pinterest , you agree to..."
4,996,Pinterest,[],every company has its terms .


In [ ]:
#Read the csv dataset as a pandas dataframe
feature_name = 'legalbert' # 'sbert' or 'legalbert'
path='/content/drive/MyDrive/Data/'+feature_name+'.pkl'
features = np.load(path, allow_pickle=True)

In [ ]:
#trim features to get ONLY training data and append embeddings
selected_features_train = features[df_train["Index"].tolist()]
selected_features_train=pd.DataFrame(selected_features_train)

dummy=selected_features_train.values.tolist()
df_train["LegalBert"]=dummy

In [ ]:
selected_features_val = features[df_val["Index"].tolist()]
selected_features_val=pd.DataFrame(selected_features_val)

dummy=selected_features_val.values.tolist()
df_val["LegalBert"]=dummy

In [ ]:
selected_features_test = features[df_test["Index"].tolist()]
selected_features_test=pd.DataFrame(selected_features_test)

dummy=selected_features_test.values.tolist()
df_test["LegalBert"]=dummy

df_test.head()

,Index,Document,Labels,Sentence,LegalBert
0,992,Pinterest,[],thank you for using pinterest !,"[0.22933529317378998, 0.10710100084543228, 0.1..."
1,993,Pinterest,[],these terms of service -lrb- `` terms '' -rrb-...,"[0.22767044603824615, 0.4018731117248535, 0.43..."
2,994,Pinterest,[],"please read these terms carefully , and contac...","[0.31588277220726013, 0.08280153572559357, 0.5..."
3,995,Pinterest,"['PINC', 'USE']","by accessing or using pinterest , you agree to...","[0.18110385537147522, 0.13075710833072662, -0...."
4,996,Pinterest,[],every company has its terms .,"[-0.18249458074569702, -0.12958329916000366, 0..."


In [ ]:
def get_closest_negative(embedding, df, label):
    query = embedding
    df_train_neg = df[~df["Labels"].str.contains(label)]

    # Stack the embeddings into a 2D array
    embeddings_stack = np.stack(df_train_neg['LegalBert'].to_numpy())

    # Calculate cosine similarity in batches
    dot_products = np.dot(query, embeddings_stack.T)
    query_norm = np.linalg.norm(query)
    features_norms = np.linalg.norm(embeddings_stack, axis=1)
    similarities = dot_products / (query_norm * features_norms)

    # Use argmax to get the index of the top similarity
    top_index = np.argmax(similarities)

    return df.iloc[top_index]


In [ ]:
def sample_n_clauses(df, n, label):
    df_pos = df[df["Labels"].apply(lambda x: label in x)]

    sampled_neg = pd.DataFrame()

    sampled_pos = df_pos.sample(n//2, replace=False)

    for index in sampled_pos.index:
        pos_sent = sampled_pos['LegalBert'][index]
        neg_sent = get_closest_negative(pos_sent, df, label)
        sampled_neg = pd.concat([sampled_neg, neg_sent.to_frame().T], ignore_index=True)

    # Reset the index of sampled_neg before concatenating
    sampled_neg = sampled_neg.reset_index(drop=True)

    result_df = pd.concat([sampled_pos, sampled_neg], axis=0, ignore_index=True)

    return result_df

result=sample_n_clauses(df_train,4,'J')
result.head()

,Index,Document,Labels,Sentence,LegalBert
0,5336,Yahoo,['J'],you and yahoo agree to submit to the personal ...,"[-0.31549254059791565, -0.2150433212518692, -0..."
1,10448,Snap,['J'],you and snap group limited consent to the excl...,"[-0.25673261284828186, -0.587177038192749, -0...."
2,6884,Tinder,[],"in doing so , you give up your right to go to ...","[0.3258098363876343, 0.12756626307964325, 0.44..."
3,6878,Tinder,[],"arbitration , class-action waiver , and jury w...","[-0.4495333135128021, 0.0821189284324646, -0.6..."


In [ ]:
################## OUTDATED ####################
#Enter the label as its capitalized acronym
#Returns a sampled df with half positives and half negatives
#No random seed because we want the sentences to change between calls for the evaluation
#We return a df for ease of future usage
# def sample_n_clauses(df=df_train, n=20, label="J"):

#     df_train_neg = df[df["Labels"].apply(lambda x: label not in x)]
#     df_train_pos = df[df["Labels"].apply(lambda x: label in x)]

#     sampled_labeled = df_train_pos.sample(n//2, replace=False)
#     sampled_other = df_train_neg.sample(n//2, replace=False)

#     # Create two new DataFrames with the sampled clauses
#     df_labeled_sampled = pd.DataFrame(sampled_labeled)
#     df_other_sampled = pd.DataFrame(sampled_other)

#     result_df = pd.concat([df_labeled_sampled, df_other_sampled], axis=0, ignore_index=False)

#     return result_df

# sampled_df= sample_n_clauses(df_train, 20)

# print(sampled_df)


In [ ]:
#Query to mistral API:
openai = OpenAI(
    api_key="XqPUITUxARnzuvDPvLWMQW86dKIAS1Ih",
    base_url="https://api.deepinfra.com/v1/openai",
)

def query_mistral(prompt, temperature=0.0, max_tokens = 50, role='user'):
    chat_completion = openai.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.1",
        messages=[{"role": role, "content": prompt},
                  {"role": "system",
                 "content": "[INST] Limit your response to 50 words.[INST]"}
            ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return chat_completion.choices[0].message.content

print(query_mistral("Hello"))

 Hello! How can I assist you today?


In [ ]:
#Compare the answer of the classification LLM to the true label to find false postives and false negatives
def is_fpnp(answer,true_label):
  if "yes" in answer.lower() and true_label=="positive":
    return "tp"
  elif "no" in answer.lower() and true_label=="positive":
    return "fn"
  elif "yes" in answer.lower() and true_label=="negative":
    return "fp"
  elif "no" in answer.lower() and true_label == "negative":
    return "tn"
  return "fn"
#Tests
print("tp expected : ",is_fpnp("YES","positive"))
print("fp expected : ",is_fpnp("YES","negative"))
print("fn expected : ",is_fpnp("NO","positive"))
print("tn expected : ",is_fpnp("NO","negative"))



tp expected :  tp
fp expected :  fp
fn expected :  fn
tn expected :  tn


In [ ]:
def get_GT(row,label_class):

  if label_class in row[2]:
    return 'positive'
  return 'negative'
#test
#print(get_GT(result.iloc[3],'J'))

In [ ]:
def get_f1(tp, fp, fn):
    f1_score = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0
    return f1_score

def get_precision(tp,fp,fn):
  if tp+fp == 0:
    return 0
  else :
    return tp/(tp+fp)

def get_recall(tp,fp,fn):
  if tp+fn==0:
    return 0
  else:
    return tp/(tp+fn)
#test
print("F1 Score (direct calculation):",get_f1(tp=10, fp=2, fn=2))


F1 Score (direct calculation): 0.8333333333333334


# Expand & Select





  #+f'The model reasons as follows: "{error["answer"]}"\n'\

In [ ]:

#Should return a list of sublist [["fp","clause","prompt","answer"],....] only return fp and fn since we won't keep the others
# Evaluation of one prompt across a minibatch, saving fn and fp prompts


def evaluate(original_prompt,minibatch_df,label):
  results=[]
  for row in minibatch_df.iterrows():
    cache={}

    true_label = ""

    if label in row[1][2]:
      true_label="positive"
    else:
      true_label="negative"

    clause=row[1][3]
    global_prompt = f'Consider the following online terms of service clause: {clause} \n {original_prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

    answer = query_mistral(global_prompt)

    status=is_fpnp(answer,true_label) #returns the tp/fp/tn/fn string


    if status == "fp" or status == "fn":
      cache["status"]=status
      cache["clause"]=clause
      cache["prompt"]=original_prompt
      cache["answer"]=answer
      cache ['gt']=true_label

      #generate critique
      if status == "fp":
        cache['prediction']="Yes"
      if status=="fn":
        cache['prediction']="No"


      results.append(cache)

      #print('RESULT DICT:',cache)

  return results



In [ ]:


def evaluate_row(original_prompt, row, label):
    cache = {}

    true_label = ""

    if label in row[1][2]:
        true_label = "positive"
    else:
        true_label = "negative"

    clause = row[1][3]
    global_prompt = f'Consider the following online terms of service clause: {clause} \n {original_prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

    answer = query_mistral(global_prompt)

    status = is_fpnp(answer, true_label)  # returns the tp/fp/tn/fn string

    if status == "fp" or status == "fn":
        cache["status"] = status
        cache["clause"] = clause
        cache["prompt"] = original_prompt
        cache["answer"] = answer
        cache['gt'] = true_label

        # generate critique
        if status == "fp":
            cache['prediction'] = "Yes"
        if status == "fn":
            cache['prediction'] = "No"

    return cache

def evaluate_parallel(original_prompt, minibatch_df, label):
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for row in minibatch_df.iterrows():
            future = executor.submit(evaluate_row, original_prompt, row, label)
            futures.append(future)

        # Wait for all futures to complete
        concurrent.futures.wait(futures)

        # Get the results
        for future in futures:
            result = future.result()
            if result:
                results.append(result)

    return results

##SPEED TEST
# sampled_df=result
# start_time = time.time()
# result=evaluate("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df,'J')
# #print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The basic evaluate function took {select_time} s to complete its task")

# start_time = time.time()
# result=evaluate_parallel("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df,'J')
# #print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized evaluate function took {select_time} s to complete its task")


In [ ]:
def get_critiq(prompt_dict, full_name_label):
  original_prompt=prompt_dict['prompt']
  prediction=prompt_dict['prediction']
  answer=prompt_dict['answer']
  true_label=prompt_dict['gt']


  critique_string= f"Online service contracts can contain clauses that may be deemed potentially unfair under European Law. Our goal is to correctly classify "+f" {full_name_label} clauses <OLD instruction>"\
                      +f"{original_prompt}<\\OLD instruction> Given instruction produced this prediction : {prediction}"\
                      +f"with this reasoning{answer}"\
                      +f", True label is: +{true_label}"\
                      +"[INST] Analyse and provide 3 constructive feedback points on the prompt. Your answer should not be longer than 50 words.[\\INST]"

  critiq=query_mistral(critique_string,temperature=1)

  return critiq


In [ ]:
########### OUTDATED ##############
#Expand function from the paper
#We first evaluate the prompt over a minibatch, saving fp and fn
# def expand(prompt,n=20,label='J',training_df=df_train):
#   print(f'Expanding the beam ..... for prompt : {prompt} \n')
#   minibatch_df = sample_n_clauses(training_df,n,label)
#   results = evaluate(prompt,minibatch_df)
#   candidates = []
#   target=""

#   fp=0
#   fn=0
#   for error in results :
#     if error["status"]=="fp":
#       fp+=1
#     elif error ["status"]=="fn":
#       fn+=1
#   print(f"Number of fp : {fp} , Number of fn : {fn}")


#   #for each fp or fn we craft a meta prompt to obtain a new (hopefully better prompt)
#   if error:
#     for error in results:
#       critiq=get_critiq(error)
#       g_meta=""
#       if error["status"]=="fp":
#         g_meta = 'The model incorrectly answers "yes", but should answer "no", as the clause is considered fair under this standard.'
#         target = "No"
#       else:
#         g_meta='The model incorrectly answers "no", but should answer "yes", as the clause is considered fair under this standard.'
#         target="Yes"
#       meta_prompt = f'Consider the following online terms of service clause: "{error["clause"]}"\n'\
#                     +f'We run this Unfairness detection instruction: <instr>{prompt}</instr>\n'\
#                     +g_meta+'\n'\
#                     +f'This instruction is critiqued here: "{critiq}"\n'\
#                     +f'[INST]Craft a short and revised instruction framed by <instr> ... </instr> tags that remedies the model\'s misunderstanding. Your instruction must not refers to the clause. End instruction on a question that leads to the correct "{target}" answer[\\INST]:\n'
#       new_prompt=query_mistral(meta_prompt,1.1)
#       candidates.append(new_prompt)
#       #print('META PROMPT:'+meta_prompt)


#       # print("number of candidates : ", len(candidates))
#       #Filtering of the results (cf Matthias Notebook)
#       filtered = []
#       for c in candidates:
#           extraction = re.findall("<instr>(.*?)</instr>", c, re.DOTALL)
#           if len(extraction) == 1:
#               filtered.append(extraction[0].strip())



#     print(f'Candidates : ',candidates)
#     print(f'filtered : ', filtered)
#   return candidates





#test

#print(expand("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?"))



In [ ]:
def expand_parallel(prompt, n, label, training_df, full_name_label):
    print(f'Expanding the beam ..... for prompt : {prompt} \n')

    minibatch_df = sample_n_clauses(training_df, n, label)
    results = evaluate_parallel(prompt, minibatch_df, label)
    candidates = []
    target = ""

    fp = 0
    fn = 0
    for error in results:
        if error["status"] == "fp":
            fp += 1
        elif error["status"] == "fn":
            fn += 1
    print(f"Number of fp : {fp} , Number of fn : {fn}")

    # for each fp or fn, craft a meta prompt to obtain a new (hopefully better prompt)
    if results:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []

            for error in results:
                critiq = get_critiq(error,full_name_label)
                g_meta = ""
                if error["status"] == "fp":
                    g_meta = 'The model incorrectly answers "yes", but should answer "no", as the clause is considered fair under this standard.'
                    target = "No"
                else:
                    g_meta = 'The model incorrectly answers "no", but should answer "yes", as the clause is considered unfair under this standard.'
                    target = "Yes"
                meta_prompt = (
                    f'Consider the following online terms of service clause: "{error["clause"]}"\n'
                    + f'We run this Unfairness detection instruction: <instr>{prompt}</instr>\n'
                    + g_meta + '\n'
                    + f'This instruction is critiqued here: "{critiq}"\n'
                    + f'[INST]Craft a short and revised instruction that remedies the model\'s misunderstanding. Your instruction must not refer to the clause. End instruction on a question that leads to the correct "{target}" answer[\\INST]:\n'
                )
                futures.append(executor.submit(query_mistral, meta_prompt, 1.1))

            # Wait for all futures to complete
            concurrent.futures.wait(futures)

            # Get the results
            for future in futures:
                new_prompt = future.result()
                candidates.append(new_prompt)

        print(f'Candidates : ', candidates)

    return candidates

#SPEED TEST

# start_time = time.time()
# result=expand("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",20,'J',sampled_df,'jurisdiction')
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The basic expand function took {select_time} s to complete its task")

# start_time = time.time()
# result=expand_parallel("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",4,'J',sampled_df,'jurisdiction')
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized selection function took {select_time} s to complete its task")

In [ ]:
#print("Eval results : ")
#print(evaluate("by accessing or signing up to receive communications , you agree to be bound by these terms .",sampled_df))
#print("---------------------------")
#print("Expand results : ")
#print(expand("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?"))

In [ ]:
############# OUTDATED ##################
# def get_score(prompt,minibatch_df,label):

#   response_dict={'tp':0,'tn':0,'fp':0,'fn':0}

#   #evaluate prompts on entire minibatch
#   for data in minibatch_df.iterrows():
#     #print(data[1][3])
#     query_string=f'Consider the following online terms of service clause: {data[1][3]}'\
#                 +f'{prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'

#     answer=query_mistral(query_string)
#     #print(answer)
#     res=is_fpnp(answer,get_GT(data[1],label))
#     response_dict[res]+=1

#   #compute F1 score

#   score=get_f1(response_dict['tp'],response_dict['fp'],response_dict['fn'])
#   print(f"Scores for prompt : {prompt} : {score} computed on {len(minibatch_df)} clauses")
#   #print(score)
#   return score

#test
#print(get_score("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df))

In [ ]:
def evaluate_data(data, prompt, label):
    query_string = f'Consider the following online terms of service clause: {data[1][3]}' \
                   + f'{prompt} \n Start your answer with "yes" or "no" and then justify your response in no more than 50 words.'
    answer = query_mistral(query_string)
    res = is_fpnp(answer, get_GT(data[1], label))

    return res

def get_score_parallel(prompt, label, minibatch_df):
    response_dict = {'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for data in minibatch_df.iterrows():
            # Pass the 'label' argument here
            future = executor.submit(evaluate_data, data, prompt, label)
            futures.append(future)

        # Wait for all futures to complete
        concurrent.futures.wait(futures)

        # Get the results
        for future in futures:
            res = future.result()
            response_dict[res] += 1

    return get_f1(response_dict['tp'],response_dict['fp'],response_dict['fn'])

#SPEED TEST

# start_time = time.time()
# result=get_score("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",sampled_df)
# print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The basic get_score function took {select_time} s to complete its task")

# start_time = time.time()
# result=get_score_parallel("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",'J',sampled_df)
# print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized get_score function took {select_time} s to complete its task")

In [ ]:
#SELECT K top performing prompts

def select(list_of_prompts,label,k,minibatchsize):

  print(f'\n SELECTING top {k} from the beam \n')
  scored_prompts={}
  if len(list_of_prompts)>0:
      minibatchsize = int(400/len(list_of_prompts))
  if minibatchsize > 56 : #The class with the least clauses in the training dataset has only 57
      minibatchsize = 56

  minibatch = sample_n_clauses(df_train, minibatchsize, label=label)


  for prompt in list_of_prompts:
    score=get_score_parallel(prompt,label,minibatch)
    scored_prompts[prompt]=score

  #sort in descending order

  sorted_scores = sorted(scored_prompts.items(), key=lambda x: x[1], reverse=True)

  scored_dict = sorted_scores
  if len(scored_dict)>=k:
    scored_dict=scored_dict[0:k]


  # Extract the top k keys
  k_best_prompts = [key for key, value in sorted_scores[:k]]



  print('\n SCORED PROMPTS:', [(key, value) for key, value in sorted(scored_prompts.items())])
  return [k_best_prompts, scored_dict]

############### Parallelized VERSION ####################### (



# def evaluate_prompt(prompt, minibatch, label):
#     return prompt, get_score(prompt, minibatch, label)

# def select_parallel(list_of_prompts, label='J', k=3, minibatchsize=20):
#     print(f'\n SELECTING top {k} from the beam \n')

#     if len(list_of_prompts)>0:
#       minibatchsize = int(400/len(list_of_prompts))
#     if minibatchsize > 56 : #The class with the least clauses in the training dataset has only 57
#       minibatchsize = 56

#     minibatch = sample_n_clauses(df_train, minibatchsize, label=label)

#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         scored_prompts = list(executor.map(evaluate_prompt, list_of_prompts, [minibatch]*len(list_of_prompts), [label]*len(list_of_prompts)))

#     # Sort in descending order based on scores
#     scored_prompts.sort(key=lambda x: x[1], reverse=True)

#     # Extract the top k prompts
#     k_best_prompts = [key for key, _ in scored_prompts[:k]]

#     print('\n SCORED PROMPTS:', scored_prompts)
#     return k_best_prompts


#test
#result=select(["Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?","Is this clause fair in that it states that any judicial proceeding must take place in a location other than the customer\'s residence?"],1)
#print(result)

#SPEED TEST

# start_time = time.time()
# result=select(["Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?","Is this clause fair in that it states that any judicial proceeding must take place in a location other than the customer\'s residence?","Under what circumstances may a clause require that a judicial proceeding be conducted in a place other than the consumer's residence?","Comprehensive instruction for a jurisdiction clause classification task: Assess whether each clause specifies a location for any legal proceedings in a manner deemed",">Does this clause require the consumer to undergo judicial proceedings in a location other than their residence, and is it considered unfair under your standard?"],'J',2,4)
# end_time = time.time()
# select_time = end_time - start_time
# print(result[0])
# print(result[1])
# print(f"The basic selection function took {select_time} s to complete its task")

# start_time = time.time()
# result=select_parallel(["Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?","Is this clause fair in that it states that any judicial proceeding must take place in a location other than the customer\'s residence?","Under what circumstances may a clause require that a judicial proceeding be conducted in a place other than the consumer's residence?","Comprehensive instruction for a jurisdiction clause classification task: Assess whether each clause specifies a location for any legal proceedings in a manner deemed",">Does this clause require the consumer to undergo judicial proceedings in a location other than their residence, and is it considered unfair under your standard?"],'J',1)
# print(result)
# end_time = time.time()
# select_time = end_time - start_time
# print(f"The parallelized selection function took {select_time} s to complete its task")

In [ ]:
import json

def save_to_file(dict_res,label,method):

  # Convert dictionary to JSON string
  json_str = json.dumps(dict_res, indent=2)

  # Specify the file path
  file_path = f"result_{label}_{method}.json"

  # Write JSON string to a text file
  with open(file_path, 'w') as file:
      file.write(json_str)

  print(f"Dictionary has been saved to {file_path}")




# Main function

In [ ]:
#main function : ALGORITHM 1

def optimization(OGprompt,beam_width,search_depth,label,full_name_label):

  result_dict = {}

  #initial prompt in beam
  beam=[OGprompt]
  print("Beam Initialization : ", beam)

  #iterate till search depth
  for i in range(0,search_depth):
    dummy=[]
    print(f'\n ################### Iteration {i+1} of {search_depth} ################\n')

    #iterrate through all prompts in beam so far
    for prompt in beam:

      #fetch candidate prompts
      results=expand_parallel(prompt,n=20,label=label,training_df=df_train,full_name_label=full_name_label)
      #print('############################',result,'############################')


      #add candidates to the beam
      if results:
        for index in range(len(results)):
          dummy.append(results[index])
          #print(f'DUMMY @ {index}',dummy[index])

    for index in range(len(dummy)):
        #print('FLAGGGG')
        beam.append(dummy[index])

    #select top k in the beam and update beam
    select_result = select(beam,label,beam_width,20)
    beam= select_result[0]
    scored_beam = select_result[1]


    result_dict[f"iteration_{i}"]=scored_beam


    print("New Beam : ",beam)

    print(f"################### END OF ITERATION {i+1} #######################")
  best_prompt = beam[0]
  result_dict["best_prompt"]=scored_beam[0]
  save_to_file(result_dict,label,"Beam_search_test_1")
  return best_prompt

#test
#result=optimization('Does this clause describe an arbitration dispute resolution process that is not fully optional to the consumer?',3,5,'A','arbitration')
#print(result)

In [ ]:
result_A=optimization('Does this clause describe an arbitration dispute resolution process that is not fully optional to the consumer?',3,5,"A","arbitration")
print(f"RESULT ARBITRATION : {result_A}")
result_CH=optimization('Does this clause specify conditions under which the service provider could amend and modify the terms of service and/or the service itself?',3,5,"CH","unilateral change to the terms")
print(f"RESULT UNILATERAL CHANGE : {result_CH}")
result_CR=optimization("Does this clause indicate conditions for content removal in the service provider's full discretion, and/or at any time for any or no reasons and/or without notice nor possibility to retrieve the content.",3,5,"CR","content removal")
print(f"RESULT CONTENT REMOVAL : {result_CR}")
result_J=optimization("Does this clause state that any judicial proceeding is to be conducted in a place other than the consumer's residence (i.e. in a different city, different country)?",3,5,"J","jurisdiction")
print(f"RESULT JURISDICTION : {result_J}")
result_LAW=optimization('Does the clause define the applicable law as different from the law of the consumer’s country of residence?',3,5,"LAW","choice of law")
print(f"RESULT CHOICE OF LAW : {result_LAW}")
result_LTD=optimization('Does this clause stipulate that duties to pay damages by the provider are limited or excluded?',3,5,"LTD","limitation of liability")
print(f"RESULT LIMITATION OF LIABILITY : {result_LTD}")
result_TER=optimization('Does this clause stipulate that the service provider may suspend or terminate the service at any time for any or no reasons and/or without notice?',3,5,"TER","unilateral termination")
print(f"RESULT UNILATERAL TERMINATION : {result_TER}")
result_USE=optimization('Does this clause stipulate that the consumer is bound by the terms of use of a specific service, simply by using the service, without even being required to mark that he or she has read and accepted them?',3,5,"USE","agreement by using")
print(f"RESULT AGREEMENT BY USING : {result_USE}")

Beam Initialization :  ['Does this clause describe an arbitration dispute resolution process that is not fully optional to the consumer?']

 ################### Iteration 1 of 5 ################

Expanding the beam ..... for prompt : Does this clause describe an arbitration dispute resolution process that is not fully optional to the consumer? 

Number of fp : 0 , Number of fn : 8
Candidates :  [' <instr>Is this arbitration dispute resolution process required by law or contract, rather than being optional for the consumer to elect participation?</instr>', ' Does this arbitration dispute resolution process provide consumers with an alternative option for resolving disputes outside of arbitration?', ' Instruction: Does the arbitration dispute resolution process described in this clause fully depend on a decision by the consumer or is it required in certain cases?\n\nPlease answer with "Yes" if it is not fully optional for the consumer. Please quote', ' How can we ensure that the language

In [ ]:
print(f"RESULT ARBITRATION : {result_A}")
print(f"RESULT UNILATERAL CHANGE : {result_CH}")
print(f"RESULT CONTENT REMOVAL : {result_CR}")
print(f"RESULT JURISDICTION : {result_J}")
print(f"RESULT CHOICE OF LAW : {result_LAW}")
print(f"RESULT LIMITATION OF LIABILITY : {result_LTD}")
print(f"RESULT UNILATERAL TERMINATION : {result_TER}")
print(f"RESULT AGREEMENT BY USING : {result_USE}")

RESULT ARBITRATION :  "Can you describe how the following clause specifies the dispute resolution process for any disagreement or claim about the terms and conditions of a service? Give at least two examples of circumstances where consumers cannot choose a private attorney general action as an option." (
RESULT UNILATERAL CHANGE :  Can you give an example of a clause that clearly outlines the conditions under which a service provider can modify or change their terms of service and/or the service itself?
RESULT CONTENT REMOVAL : Does this clause indicate conditions for content removal in the service provider's full discretion, and/or at any time for any or no reasons and/or without notice nor possibility to retrieve the content.
RESULT JURISDICTION :  Can the model determine if a clause specifies that any legal proceeding must take place in a jurisdiction different from the user's home country, and if so, is this clause considered unfair?
RESULT CHOICE OF LAW :  "Evaluate the language o